<a href="https://colab.research.google.com/github/gayatri-sa/ml_python_2019/blob/master/03-09-Cross%20Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cross Validation (CV)

* Hold out Cross Validation
* k-fold Cross Validation

A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. 

In the basic approach, called k-fold CV, the training set is split into k smaller sets. The following procedure is followed for each of the k “folds”:
* A model is trained using k-1 of the folds as training data;
* the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. 

# Holdout Method

* Split initial dataset into a separate training and test dataset
* Training dataset - model training
* Test dataset - estimate its generalisation performance

<img src='https://github.com/gayatri-sa/ml_python_2019/blob/master/img/cv_holdout_method.PNG?raw=true' width=50%>


A variation is to split the training set to two :- training set and validation set

Training set:- For fitting different models

Validation set :- For tuning and comparing different parameter settings to further improve the performance for making predictions on unseen data. And finally for model selection.

This process is called model selection. We want to select the optimal values of tuning parameters (also called hyperparameters). 


<img src='https://github.com/gayatri-sa/ml_python_2019/blob/master/img/cv_holdout_w_validation.PNG?raw=true' width=50%>

# K-fold Cross-validation

* Randomly split the training dataset into k folds without replacement.

* k — 1 folds are used for the model training.

* The one fold is used for performance evaluation. 

This procedure is repeated k times. 

Final outcomes:- k models and performance estimates.

* calculate the average performance of the models based on the different, independent folds to obtain a performance estimate that is less sensitive to the sub-partitioning of the training data compared to the holdout method. 

* k-fold cross-validation is used for model tuning. Finding the optimal hyperparameter values that yields a satisfying generalization performance.

* Once we have found satisfactory hyperparameter values, we can retrain the model on the complete training set and obtain a final performance estimate using the independent test set. The rationale behind fitting a model to the whole training dataset after k-fold cross-validation is that providing more training samples to a learning algorithm usually results in a more accurate and robust model.


* Common k is 10

* For relatively small training sets, increase the number of folds. 

<img src='https://github.com/gayatri-sa/ml_python_2019/blob/master/img/cv_cross_validation.PNG?raw=true' width=50%>

## Stratified k-fold cross-validation

* variation of k-fold
* Can yield better bias and variance estimates, especially in cases of unequal class proportions

***

# Illustration

## Cross-validation: evaluating estimator performance

Adapted from [scikit learn](Cross-validation: evaluating estimator performance)

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called **overfitting**. 

To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a **test set** `X_test`, `y_test`. Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally.

In scikit-learn a random split into training and test sets can be quickly computed with the `train_test_split` helper function. Let’s load the iris data set to fit a linear support vector machine on it:

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

boston = datasets.load_boston()
boston.data.shape, boston.target.shape

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


((506, 13), (506,))

In [3]:
print(np.__version__)
print(pd.__version__)
import sys
print(sys.version)
import sklearn
print(sklearn.__version__)

1.18.2
1.0.3
3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
0.22.2.post1


We can now quickly sample a training set while holding out 40% of the data for testing (evaluating) our regressor:

In [4]:
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.4, random_state=0)

print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)

regression = svm.SVR(kernel='linear', C=1).fit(X_train, y_train)
regression.score(X_test, y_test)      

(303, 13) (303,)
(203, 13) (203,)


0.667431382173115

When evaluating different settings (“hyperparameters”) for estimators, such as the `C` setting that must be manually set for an SVM, there is still a risk of overfitting on the test set because the parameters can be tweaked until the estimator performs optimally. 

This way, knowledge about the test set can “leak” into the model and evaluation metrics no longer report on generalization performance. 

To solve this problem, yet another part of the dataset can be held out as a so-called “validation set”: training proceeds on the training set, after which evaluation is done on the validation set, and when the experiment seems to be successful, final evaluation can be done on the test set.

However, by partitioning the available data into three sets, we drastically reduce the number of samples which can be used for learning the model, and the results can depend on a particular random choice for the pair of (train, validation) sets.



A solution to this problem, as discussed earlier, is a procedure called cross-validation (CV for short). A test set should still be held out for final evaluation, but the validation set is no longer needed when doing CV. In the basic approach, called k-fold CV, the training set is split into k smaller sets (other approaches are described below, but generally follow the same principles). The following procedure is followed for each of the k “folds”:

* A model is trained using k-1 of the folds as training data;
* the resulting model is validated on the remaining part of the data (i.e., it is used as a test set to compute a performance measure such as accuracy).

The performance measure reported by k-fold cross-validation is then the average of the values computed in the loop. This approach can be computationally expensive, but does not waste too much data (as it is the case when fixing an arbitrary test set), which is a major advantage in problem such as inverse inference where the number of samples is very small.

## Computing cross-validated metrics

In [5]:
from sklearn.model_selection import cross_val_score
regression = svm.SVR(kernel='linear', C=1)
scores = cross_val_score(regression, boston.data, boston.target, cv=5)
scores        

array([0.77285459, 0.72771739, 0.56131914, 0.15056451, 0.08212844])

The mean score and the 95% confidence interval of the score estimate are hence given by:

In [6]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

Accuracy: 0.46 (+/- 0.29)


By default, the score computed at each CV iteration is the score method of the estimator. It is possible to change this by using the scoring parameter:

In [7]:
from sklearn import metrics
scores = cross_val_score(regression, boston.data, boston.target, cv=5, scoring='neg_mean_squared_error')
scores 

array([ -7.84451123, -24.78772444, -35.13272326, -74.50555945,
       -24.40465975])

See [The scoring parameter: defining model evaluation rules](http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) for details. In the case of the Iris dataset, the samples are balanced across target classes hence the accuracy and the F1-score are almost equal.

When the `cv` argument is an integer, `cross_val_score` uses the `KFold` or `StratifiedKFold` strategies by default, the latter being used if the estimator derives from ClassifierMixin.

## K-fold

`KFold` divides all the samples in k groups of samples, called folds (if k = n, this is equivalent to the Leave One Out strategy), of equal sizes (if possible). The prediction function is learned using k - 1 folds, and the fold left out is used for test.

Example of 2-fold cross-validation on a dataset with 4 samples:

In [0]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print("%s %s" % (train, test))

[2 3] [0 1]
[0 1] [2 3]


## Stratified k-fold

StratifiedKFold is a variation of k-fold which returns stratified folds: each set contains approximately the same percentage of samples of each target class as the complete set.

Example of stratified 3-fold cross-validation on a dataset with 10 samples from two slightly unbalanced classes:

In [0]:
from sklearn.model_selection import StratifiedKFold

X = np.ones(10)
y = [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print("%s %s" % (train, test))

[2 3 6 7 8 9] [0 1 4 5]
[0 1 3 4 5 8 9] [2 6 7]
[0 1 2 4 5 6 7] [3 8 9]


In [0]:
X

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [0]:
y

[0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
#from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.pipeline import make_pipeline
#pipe_lr = make_pipeline(StandardScaler(),
#                        PCA(n_components=2),
#                        LogisticRegression(random_state=1))
pipe_svm = make_pipeline(StandardScaler(),
                         PCA(n_components=2),
                         svm.SVR(kernel='linear', C=1))
pipe_svm.fit(X_train, y_train)
y_pred = pipe_svm.predict(X_test)
print('Test Accuracy: %.3f' % pipe_svm.score(X_test, y_test))

Test Accuracy: 0.391


In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator=pipe_svm,
                         X=X_train,
                         y=y_train,
                         cv=10,
                         n_jobs=1)
print('CV accuracy scores: %s' % scores)


CV accuracy scores: [0.63971176 0.43579197 0.46977821 0.25027246 0.5124364  0.26221374
 0.30877195 0.54528563 0.37810066 0.47313549]


In [0]:
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores),
                                      np.std(scores)))

CV accuracy: 0.428 +/- 0.121


***